In [ ]:
import numpy as np
import pandas as pd
import random
import torch

seed = 42
np.random.seed(seed)
random.seed(seed)
torch.manual_seed(seed)

In [2]:
from datasets import load_dataset

In [3]:
from datasets import load_dataset

# Login using e.g. `huggingface-cli login` to access this dataset
ds = load_dataset("friedrichor/MSR-VTT", "train_9k")['train']

In [4]:
print(ds)

Dataset({
    features: ['video_id', 'video', 'caption', 'source', 'category', 'url', 'start time', 'end time', 'id'],
    num_rows: 9000
})


In [5]:
print(ds.column_names)
print(ds.features)

['video_id', 'video', 'caption', 'source', 'category', 'url', 'start time', 'end time', 'id']
{'video_id': Value('string'), 'video': Value('string'), 'caption': List(Value('string')), 'source': Value('string'), 'category': Value('int64'), 'url': Value('string'), 'start time': Value('float64'), 'end time': Value('float64'), 'id': Value('int64')}


In [6]:
import cv2
import mat73
import numpy as np
fps_target = 2

In [10]:
import subprocess

def download_clip(url, start, end, out_path):
    # Download and trim using ffmpeg
    
    cmd = [
        "yt-dlp", "-f", "bestvideo*+bestaudio/best",
        "--download-sections", f"*{start}-{end}",
        "-o", out_path, url
    ]
    subprocess.run(cmd, check=True)


In [ ]:
sampled_ds = ds.select()

In [8]:
test_url = ds[0]['url']
start = ds[0]['start time']
end = ds[0]['end time']
print(test_url)
print(start, end)
print(ds[0]['caption'])

https://www.youtube.com/watch?v=9lZi22qLlEo
137.72 149.44
['a car is shown', 'a group is dancing', 'a man drives a vehicle through the countryside', 'a man drives down the road in an audi', 'a man driving a car', 'a man is driving a car', 'a man is driving down a road', 'a man is driving in a car as part of a commercial', 'a man is driving', 'a man riding the car speedly in a narrow road', 'a man showing the various features of a car', 'a man silently narrates his experience driving an audi', 'a person is driving his car around curves in the road', 'a person telling about a car', 'guy driving a car down the road', 'man talking about a car while driving', 'the man drives the car', 'the man driving the audi as smooth as possible', 'a man is driving', 'guy driving a car down the road']


In [11]:
download_clip(test_url, start, end, "test.mp4")

In [ ]:
'''

'''